<a id="intro"></a>
<br>
<br>
<br>

# Calculate Window Centroid & Obstruction Angle
This notebook is dedicated to computing the centroid of rectangular windows on 3D buildings as well as their corresponding obstrucion angle.


The notebook is divided into the following parts:
- [Import modules](#in_modules)
- [Set environment & input variables](#set_vars)
- [Help functions](#help_funcs)
- [Compute window cenroids](#w_centroids)
- [Compute obstruction angle (OA)](#obstr_angle)
- [Main function](#main_func)
- [Execute calculations](#exe_code)
- [Conrol output](#control_out)
- [Reference links](#references)
<br>
<br>
<br>

<a id="in_modules"> </a>
<br>
<br>

## Import modules

In [1]:
#Import modules:
import arcpy
import os
import math

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="set_vars"></a>
<br>
<br>

## Set environment & input variables

In [2]:
#Set help variables:
rooftype = "takkant" #roof type ["taknock", "medeltak", "takkant", "lu_mun"]
b_number = str(3)    #building number [1,.., 10, 12, ..., 16]


# Set environment settings:
working_environment = r"C:\Users\Karolina.Pantazatou\Documents\ArcGIS\Projects\LU_DSM\OA_results\inermediate_files"
arcpy.env.workspace = working_environment
arcpy.env.overwriteOutput = True

# Set the outputZFlag environment to Enabled
arcpy.env.outputZFlag = "Enabled"

#Set centroid input variables:
c_in_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\LU_DSM\\w_groups\\"
w_fc = c_in_path + "boi"+ b_number +"_w.shp" #window feature class
VCS = 5613          #vertical coordinate system (5613 corresponds to RH2000) 

#Set centroid output variables:
c_out_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\LU_DSM\\w_centroids\\"
c_out_file = "boi"+ b_number +"_w_centroids.shp"
c_output = c_out_path + c_out_file

#Set DSM input variables:
dsm_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\LU_DSM\\clipped_DSMs\\lu_mun_2d_fp\\"+rooftype+"\\"
dsm_file = "group"+ b_number +"_dsm.tif"
dsm_fc = dsm_path + dsm_file

### Set the 2D building fooprint input variables ###
#Check if input DSMs were created from buildings in the Lund Municipality 3D city model
#if rooftype=="lu_mun":
#    b_fp_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\LU_DSM\\LU_MUN_b_footprints\\"
#    b_fp_file = "lund_mun_b_fp.shp"
#    b_fp_fc = b_fp_path + b_fp_file

#If the input DSMs were created using the Lantmäteriet 2D building footprints from fastighetskartan: 
#else:
#    b_fp_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\LU_DSM\\LM_footprints\\"
#    b_fp_file = "lund_b_selection.shp"
#    b_fp_fc = b_fp_path + b_fp_file
b_fp_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\Lund_roof_heights\\neighbourhoods_2D\\"  
b_fp_file = "all_b_2d.shp"
b_fp_fc = b_fp_path + b_fp_file

#Viewshed out:
vs_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\Testing_OA\\viewshed_out\\"

#Set OA output variables:
#oa_out_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\LU_DSM\\OA_results\\"+rooftype+"\\"
#oa_out_file = "boi"+ b_number +"_w_oa.shp"
#oa_output = oa_out_path + oa_out_file

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="help_funcs"></a>
<br>
<br>

## Help functions

In [3]:
def get_field_names(fc):
    
    #Get a list of fieldnames for input shapefile:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Return list with fieldnames:
    return f_names

In [4]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) and returns
#the content of the corresponding Table Of Contents (TOC):
def get_TOC(fc):
    
    #Get a list of field-names:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Open shapefile and print every row in TOC:
    with arcpy.da.SearchCursor(fc, f_names) as cursor:
        
        for row in cursor:
            
            print(row)

In [5]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) together with 
#the column name whose values we want to extract and a boolean 
#(unique=True returns only unique values in column) and returns
#the content of the corresponding column in a list:
def get_TOC_column(fc, colname, unique):
    
    #Create a list to store the column values:
    ls = []
    
    #Open shapefile:
    with arcpy.da.SearchCursor(fc, colname) as cursor:
        
        #Loop through every row in shapefile:
        for row in cursor:
            
            #If user wants the output list to only include unique values:
            if (unique):
                
                #Check that values is not already stored in the list:
                if row[0] not in ls:
                    
                    #Append value:
                    ls.append(row[0])
                    
            else:
                
                #Append column value to list:
                ls.append(row[0])
    
    #return list:
    return ls

In [6]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) together with 
#the column names whose values we want to extract and a boolean 
#(unique=True returns only unique values in row) and returns
#the content of the corresponding columns in a list:
def get_TOC_columns(fc, colnames, unique):
    
    #Create a list to store the column values:
    ls = []
    
    #Open shapefile:
    with arcpy.da.SearchCursor(fc, colnames) as cursor:
        
        #Loop through every row in shapefile:
        for row in cursor:
            
            #If user wants the output list to only include 
            #unique combination of row values:
            if (unique):
                
                #Check that row values are not already stored in the list:
                if row not in ls:
                    
                    #Append row values:
                    ls.append(row)
                    
            else:
                
                #Append row value to list:
                ls.append(row)
    
    #return list:
    return ls

In [7]:
#Function that creates a new numeric field/column for a given 
#feature class 
#(feature_class-string, colname-string, datatype-string ("LONG"), numofvalues-integer, numofdecvalues-integer)
def create_long_field(fc, colname, datatype, numofvalues, numofdecvalues):
    
    #Extract field-names from shapefile:
    fieldnames = [f.name for f in arcpy.ListFields(fc)]
    
    #Check if "NN_dist" field already exists:
    if(colname not in fieldnames):

        #Add new field (distance to nearest neighbor) to shapefile: 
        try:
            arcpy.AddField_management(fc, colname, datatype, numofvalues, numofdecvalues)
            #print(colname+ "-field added to " + fc[0])

        except Exception as e:
            print("Error: " + e.args[0])
    
    
    

In [8]:
#Function that reads and returns the name of the spatial reference
#system for the input feature layer:
def get_CRS(fc):
    
    #Extract spatial reference system information from feature class:
    spatial_ref = arcpy.Describe(fc).spatialReference
    
    #Return:
    return spatial_ref.name
    

In [9]:
#Function that takes a string describing where a
#raster is stored, checks if it is empty and returns
#True or False accordingly:
def check_empty_raster(r_path):
    
    #Read raster:
    raster = arcpy.Raster(r_path)
    
    #Get raster maximum value:
    max_val = arcpy.sa.Raster(raster).maximum
    
    #If the raster is empty:
    if max_val is None:
        
        #Set check to true:
        raster_check = True
    
    #If raster is not empty:
    else:
        
        #Set raster to false:
        raster_check = False
        
    #Return check value:
    return raster_check

In [10]:
#Function that takes a string describing the path 
#to a directory as input and deletes all files, 
#subdirectories, and symbolic links from that directory:
def delete_dir_files(dir):
    
    #Import modules: 
    import os, glob
    
    #Loop through all files in given directory:
    for file in os.listdir(dir):
        
        #Find temp help-files:
        if (file.endswith(".lock")==False):
            
            #Delete file:
            arcpy.Delete_management(file)

In [11]:
#Function that takes a string representing the path
#to a shapefile as input, checks if it is empty and
#returns a boolean value accordingly:
def shape_is_empty(fc):
    
    #Check if input is a string:
    if(isinstance(fc, str)):
        
        try:
            
            #Check if shapefile is empty:
            if arcpy.management.GetCount(fc)[0]=="0":
                
                return True
            
            #If shapefile is not empty:
            else:
                return False
        
        #If the path to the shapefile is not valid:
        except:
            print('\nError!\nCould not find file...')
            return None
            
    #If the function input is not a string:    
    else:
        print('\nInvalid input!\nExpected a string represnting the path to a shapefile...')
        return None
 

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="w_centroids"></a>

<br>
<br>

## Compute window centroids

In [12]:
#Function that takes a multipatch layer of rectangular windows as input 
#and returns a new feature class with their centroids (point layer):
def compute_w_centroid(fc, vcs, out_file):
    
    #Convert window multipatch to points:
    w_points = arcpy.FeatureVerticesToPoints_management(fc)
    
    #Convert window points to lines:
    w_lines = arcpy.management.PointsToLine(Input_Features=w_points, Line_Field="ORIG_FID",
                                            Sort_Field="", Close_Line="NO_CLOSE")
    
    #Extract 3D geometry (Z: elevation for every window line):
    arcpy.CheckExtension("3D") 
    arcpy.CheckOutExtension("3D")
    arcpy.ddd.AddZInformation(w_lines, "Z_MEAN", 'NO_FILTER')
    arcpy.CheckInExtension("3D")
    
    #Add centroid fields to window-lines shapefile:
    create_long_field(w_lines, "C_X_coord", "LONG", 12, 3)
    create_long_field(w_lines, "C_Y_coord", "LONG", 12, 3)
    
    
    #Call function to calculate centroid:
    arcpy.management.CalculateGeometryAttributes(in_features=w_lines,
                                                 geometry_property=[["C_X_coord", "CENTROID_X"],
                                                                    ["C_Y_coord", "CENTROID_Y"]],
                                                 length_unit="", area_unit="",
                                                 coordinate_system=arcpy.SpatialReference(arcpy.Describe(fc).spatialReference.factoryCode, vcs),
                                                 coordinate_format="SAME_AS_INPUT")
    
    #Create separate shapefile (point layer) with window centroids:
    arcpy.management.XYTableToPoint(w_lines, out_file,
                                    x_field = "C_X_coord", 
                                    y_field = "C_Y_coord",
                                    z_field = "Z_MEAN",
                                    coordinate_system = arcpy.SpatialReference(arcpy.Describe(fc).spatialReference.factoryCode, vcs))
    
    
    #Rename field corresponding to centroid c-coordinate:
    create_long_field(out_file, "C_Z_coord", "LONG", 12, 3)
    arcpy.CalculateField_management(out_file, "C_Z_coord", "!Z_MEAN!", "PYTHON_9.3", "")
    arcpy.DeleteField_management(out_file, "Z_MEAN")
    arcpy.DeleteField_management(out_file, "Id")
    
    #Join input feature class with output feature class to maintain columns:
    arcpy.JoinField_management(out_file, 'ORIG_FID', fc, 'FID')
    
    #Delete help-files:
    #Check files in working folder:
    for file in os.listdir(working_environment):

        #Find temp help-files:
        if (file.endswith(".shp") & ("FeatureVerticesToPoin" in file) & (fc.split('.')[0] in file)):

            #Delete files:
            arcpy.Delete_management(file)


<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="obstr_angle"></a>

<br>
<br>

## Compute obstruction angle

<br>
<br>

- Get top 2 window vertices

- Run Near3D for each one

- Compute horizontal start/end angle for both sets of Near_angle_H

- Transform horizontal start/end angles to viewshed2-tool orientation system

- Run viewshed2 tool for both options

- Check viewshed2 tool output for both previous options 
    - if it is possible to use this, continue with workflow from notebook
    - if it is not possible to decide on orientation of window based on this, do the following

- Run Near-tool (ANGLE-checked, LOCATION-checked) between window centroid and visible observation points (both sets - if available).

- If distance is closer than x meters discard this dataset.

- Compare Near_angle (between centroid & observation datasets) with Near_angle_H (between window vertices). Their difference should be about 90 degrees.


In [13]:
#Function that takes as input a window shapefile & adds a
#field for storing the obstruction angle  (OA) for every window-feature:
def add_OA_field(fc):
    
    #Extract field-names from shapefile:
    fieldnames = [f.name for f in arcpy.ListFields(fc)]
    
    #Check if "NN_dist" field already exists:
    if('OA' not in fieldnames):

        #Add column to window fc for storing obstruction angle value:
        try:
            
            arcpy.AddField_management(in_table=fc, field_name='OA', field_type='LONG', field_precision=9, field_scale=2, field_is_nullable='NULLABLE')
            print("OA field added to " + fc)

        except Exception as e:
            print("Error: " + e.args[0])

In [14]:
def extract_rectangle_top_vertices(fc):
    
    #Extract rectangle (window) vertices:
    w_p = arcpy.management.FeatureVerticesToPoints(in_features=fc, point_location="ALL")
            
    #Add elevation-info to rectangle vertices:
    arcpy.CheckExtension("3D") 
    arcpy.CheckOutExtension("3D")
    w_p_z_info = arcpy.ddd.AddZInformation(in_feature_class=w_p[0], out_property=["Z"])
    arcpy.CheckInExtension("3D")
    
    #print()
    #print(get_field_names(w_p_z_info))
    #print('w_p_z_info: ', get_TOC_columns(w_p_z_info, get_field_names(w_p_z_info), False))
    #print()
            
    #Delete Identical entries:
    arcpy.DeleteIdentical_management(w_p_z_info, ('Shape','Z'))
    
    #print('After deleting duplicates: ', get_TOC_columns(w_p_z_info, get_field_names(w_p_z_info), False))
            
    #Extract top 2 rectangle vertices of same height:
    max_vertices = arcpy.management.SelectLayerByAttribute(in_layer_or_view=w_p_z_info,
                                                           selection_type="NEW_SELECTION",
                                                           where_clause="Z <= "+str(max(get_TOC_column(w_p_z_info,
                                                                                                      ('Z'), True))+0.1)+" And Z >= "+str(max(get_TOC_column(w_p_z_info,
                                                                                                      ('Z'), True))-0.1), 
                                                           invert_where_clause="")
    
    #print('max_vertices: ', get_TOC_columns(max_vertices, get_field_names(max_vertices), False))
    
    
    #Check that feature class includes only 2 entries:
    if (len(get_TOC_columns(max_vertices[0], get_field_names(max_vertices[0]), False))==2):
        
        #Return feature class with top 2 vertices:
        return max_vertices
    
    else:
        print('Window is not a rectangle! It was not possible to find 2 points of the same max height.\n\n')
        
        return None
    
    

In [15]:
def check_minus_180(n):
    
    result = n
    
    #Check if number is smaller than -180:
    if n<(-180):
        
        #Add 360:
        result = n + 360.0
    
    #Return number:
    return result
        

In [16]:
def check_plus_180(n):

    result = n
    
    #Check if number is higher than 180:
    if n>180:
        
        #Subtract 360:
        result = n - 360.0
    
    #Return number:
    return result

In [17]:
def nearAngleH_2_viewshedAngles(angle):
    
    if ((angle>=-180) & (angle<=-90)):
        
        angle = abs(angle + 90)
        
        
    elif ((angle>-90) & (angle<=0)):
        
        angle = 270 + abs(angle)
        
        
    elif (angle>=90) & (angle<180):
        
        angle = abs(angle - 180) + 90
        
    
    elif (angle>0) & (angle<90):
        
        angle = 270 - angle
        
    else:
        print('Invalid angle value for conversion')
        value = -9999
        
    return angle

In [18]:
def check_angle_diff(a1, a2, a3, a4): 
    
    #Pairs must differ by 10 degrees 
    #(we look for obstrucion angle between 85 and 95 degrees from every centroid):
    diff1 = abs(a1 - a3)
    diff2 = abs(a1 - a4)
    diff3 = abs(a2 - a3)
    diff4 = abs(a2 - a4)
    
    #Create lists to store pairs of horizontal start- & ending-values:
    horiz_angle_values_1 = []
    horiz_angle_values_2 = []
    
    #Add angles that differ by 10 degrees in same list:
    if ((diff1>=9.5) & (diff1<=10.9)):
        horiz_angle_values_1.append(a1)
        horiz_angle_values_1.append(a3)
    
    if ((diff2>=9.5) & (diff2<=10.9)):
        horiz_angle_values_1.append(a1)
        horiz_angle_values_1.append(a4)
    
    if ((diff3>=9.5) & (diff3<=10.9)):
        horiz_angle_values_2.append(a2)
        horiz_angle_values_2.append(a3)
    
    if ((diff4>=9.5) & (diff4<=10.9)):
        horiz_angle_values_2.append(a2)
        horiz_angle_values_2.append(a4)
        
    if((a1>=350) & (a3<10) & ((360 - a1 + a3)>=9.5) | ((360 - a1 + a3)<=10.9)):
        horiz_angle_values_1.append(a1)
        horiz_angle_values_1.append(a3)
        
    if((a1>=350) & (a4<10) & ((360 - a1 + a4)>=9.5) | ((360 - a1 + a4)<=10.9)):
        horiz_angle_values_1.append(a1)
        horiz_angle_values_1.append(a4)
        
    if((a3>=350) & (a1<10) & ((360 - a3 + a1)>=9.5) | ((360 - a3 + a1)<=10.9)):
        horiz_angle_values_1.append(a1)
        horiz_angle_values_1.append(a3)
        
    if((a4>=350) & (a1<10) & ((360 - a4 + a1)>=9.5) | ((360 - a4 + a1)<=10.9)):
        horiz_angle_values_1.append(a1)
        horiz_angle_values_1.append(a4)
        
    ##
    if((a2>=350) & (a3<10) & ((360 - a2 + a3)>=9.5) | ((360 - a2 + a3)<=10.9)):
        horiz_angle_values_2.append(a2)
        horiz_angle_values_2.append(a3)
        
    if((a2>=350) & (a4<10) & ((360 - a2 + a4)>=9.5) | ((360 - a2 + a4)<=10.9)):
        horiz_angle_values_2.append(a2)
        horiz_angle_values_2.append(a4)
        
    if((a3>=350) & (a2<10) & ((360 - a3 + a2)>=9.5) | ((360 - a3 + a2)<=10.9)):
        horiz_angle_values_2.append(a2)
        horiz_angle_values_2.append(a3)
        
    if((a4>=350) & (a2<10) & ((360 - a4 + a2)>=9.5) | ((360 - a4 + a2)<=10.9)):
        horiz_angle_values_2.append(a2)
        horiz_angle_values_2.append(a4)
        
    
    #print('diff1: ', diff1)
    #print('diff2: ', diff2)
    #print('diff3: ', diff3)
    #print('diff4: ', diff4)
    
    return horiz_angle_values_1, horiz_angle_values_2    

In [19]:
#Function that takes a fetature class with two points as input and
#returns their horizontal angle as calculated using Near 3D twice.
#Once from pointA to pointB & once from pointB to pointA:
def get_near_angle_h(fc):
    
    #Create 2 separate feature layers for every window vertex:
    arcpy.management.SelectLayerByAttribute(in_layer_or_view=fc,
                                            selection_type="NEW_SELECTION",
                                            where_clause="FID = {}".format(min([i[0] for i in get_TOC_columns(fc[0], 
                                                                                                   get_field_names(fc[0]), False)])))
    #Write selected features to a new feature class:
    if arcpy.Exists('v1.shp'):
        arcpy.Delete_management('v1.shp')
        arcpy.CopyFeatures_management(fc, 'v1.shp')
    else:
        arcpy.CopyFeatures_management(fc, 'v1.shp')
                
            
    arcpy.management.SelectLayerByAttribute(in_layer_or_view=fc,
                                            selection_type="NEW_SELECTION",
                                            where_clause="FID = {}".format(max([i[0] for i in get_TOC_columns(fc[0],
                                                                                                              get_field_names(fc[0]), False)])))
            
    #Write selected features to a new feature class:
    if arcpy.Exists('v2.shp'):
        arcpy.Delete_management('v2.shp')
        arcpy.CopyFeatures_management(fc, 'v2.shp') 
    else:
        arcpy.CopyFeatures_management(fc, 'v2.shp') 
            
            
    #print()
    #print(get_field_names('v1.shp'))
    #print('v1: ', get_TOC_columns('v1.shp', get_field_names('v1.shp'), False))
    #print('v2: ', get_TOC_columns('v2.shp', get_field_names('v2.shp'), False))
    #print()

    #Handle license issues for 3D:
    arcpy.CheckExtension("3D") 
    arcpy.CheckOutExtension("3D")
    arcpy.env.outputZFlag = "Enabled"

    #Run 3D Near to calculate horizontal angle between both vertices:
    arcpy.ddd.Near3D(in_features='v1.shp', near_features=['v2.shp'], search_radius="", angle="ANGLE")
    arcpy.ddd.Near3D(in_features='v2.shp', near_features=['v1.shp'], search_radius="", angle="ANGLE")

    #Handle license issues for 3D:
    arcpy.CheckInExtension("3D")
            

    #print()
    #print('After running Near 3D')
    #print(get_field_names('v1.shp'))
    #print('v1: ', get_TOC_columns('v1.shp', get_field_names('v1.shp'), False))
    #print('v2: ', get_TOC_columns('v2.shp', get_field_names('v2.shp'), False))
    #print()


    #Get NEAR_ANG_H for the couple v1 & v2:
    nah1 = get_TOC_column('v1.shp', 'NEAR_ANG_H', False)[0]

    #Get NEAR_ANG_H for the couple v2 & v1:
    nah2 = get_TOC_column('v2.shp', 'NEAR_ANG_H', False)[0]
            
    
    #Return horizontal angles between points A-B & B-A:
    return nah1, nah2

In [20]:
#Function that takes a list as input and
#checks if it contains nubers than are higher
#than a given number:
def check_high_val(ls, n):
    
    #Create check variable:
    check = False
    
    #Loop through all list items:
    for num in ls:
        
        #Check if item value is higher than given number:
        if num > n:
            
            #If list contains item whose value is greater
            #than given number, set check variable to True:
            check = True
    
    #Return check variable:
    return check
            

In [21]:
#Funcion that takes a feature class as input (includes two points - top 2 window vertices)
#and returns 2 lists with suggestions for the start- and end- horizontal angle fields in
#the viewshed2 3D visibility tool:
def get_viewshed_horiz_angles(fc):
    
    #Get h angles:
    nah1, nah2 = get_near_angle_h(fc)
    
    #Get horizontal angle candidates for nah1:
    hac1a = nah1 + 85.0
    hac1b = nah1 - 85.0

    #Get horizontal angle candidates for nah2:
    hac2a = nah2 + 85.0
    hac2b = nah2 - 85.0

    #Check that results do not exceed 180 or are less than -180 degrees:
    hac1a = check_minus_180(hac1a)
    hac1a = check_plus_180(hac1a)
    hac1b = check_minus_180(hac1b)
    hac1b = check_plus_180(hac1b)
    hac2a = check_minus_180(hac2a)
    hac2a = check_plus_180(hac2a)
    hac2b = check_minus_180(hac2b)
    hac2b = check_plus_180(hac2b)

    #Transform degrees to from North=-90, East=+180 or -180, South= 90, West=0 to 
    #viewshed coord system North=0 0r 360, East=90, South=180, West=270:
    conv_angle1 = nearAngleH_2_viewshedAngles(hac1a)
    conv_angle2 = nearAngleH_2_viewshedAngles(hac1b)
    conv_angle3 = nearAngleH_2_viewshedAngles(hac2a)
    conv_angle4 = nearAngleH_2_viewshedAngles(hac2b)

    
    
    #print('nah1: ', nah1)
    #print('nah2: ', nah2)
    #print()
    #print('hac1a: ', hac1a)
    #print('hac1b: ', hac1b)
    #print('hac2a: ', hac2a)
    #print('hac2b: ', hac2b)
    #print()
    #print('conv_angle1: ', conv_angle1)
    #print('conv_angle2: ', conv_angle2)
    #print('conv_angle3: ', conv_angle3)
    #print('conv_angle4: ', conv_angle4)
    
    #Call function to get angle difference:
    horiz_angle_values_1, horiz_angle_values_2 = check_angle_diff(conv_angle1, conv_angle2, conv_angle3, conv_angle4)
   

    print()
    print('pair 1: ', horiz_angle_values_1)
    print('pair 2: ', horiz_angle_values_2)
   

    #Check order of values for option 1:
    if (check_high_val(horiz_angle_values_1, 350)):
        
        horiz_angle_values_1 = sorted(horiz_angle_values_1, key=lambda x: (x>350, x), reverse=True)
    
    else:
        horiz_angle_values_1.sort(reverse=False)
    
    #Check order of values for option 2:
    if (check_high_val(horiz_angle_values_2, 350)):
        
        horiz_angle_values_2 = sorted(horiz_angle_values_2, key=lambda x: (x>350, x), reverse=True)
    
    else:
        horiz_angle_values_2.sort(reverse=False) 

    
    #Return suggestions for start- & end-horizontal angle values:
    return horiz_angle_values_1, horiz_angle_values_2

In [22]:
#Function that extracts cells from a raster based on another raster (mask):
def extract_by_mask(inRaster, inMaskData):
    
    #Execute ExtractByMask tool:
    arcpy.CheckExtension('spatial') 
    arcpy.CheckOutExtension('spatial')
    outExtractByMask = arcpy.sa.ExtractByMask(inRaster, inMaskData)
    arcpy.CheckInExtension('spatial')
    
    #Return raster with visible pixels:
    return outExtractByMask
        

In [23]:
#Function that writes selected features to a
#new feature class:
def save_as_fc(selected_features, out_fc_file):
    
    #Check if output fc file already exists:
    if arcpy.Exists(out_fc_file):
        
        #Delete existing file:
        arcpy.Delete_management(out_fc_file)
        
        #Write selected features to a new feature class:
        arcpy.CopyFeatures_management(selected_features, out_fc_file)
        
    else:
        
        #Write selected features to a new feature class:
        arcpy.CopyFeatures_management(selected_features, out_fc_file)
    
    

In [24]:
#Function that takes two 3D points (tuples of (x-coord, y-coord, z-cood))
#as input and returns their 3D Euclidean distance: 
def point_3D_dist(p1, p2):
    
    #Create and initialize variable to store 3D distance:
    dist_3D = -1

    try:
        
        #Compute distance:
        dist_3D = math.sqrt(pow(p2[0] - p1[0], 2) + pow(p2[1] - p1[1], 2) + pow(p2[2] - p1[2], 2))
    
    
    #Print error message if distance cannot be calculated:
    except:
        
        print("3D Euclidean distance could not be calculated...")
    
    #Return distance value:
    return dist_3D

In [25]:
#Function that takes two points (tuples of (x-coord, y-coord, z-cood))
#as input and returns their 2D Euclidean distance: 
def point_2D_dist(p1, p2):
    
    #Create and initialize variable to store 3D distance:
    dist_2D = -1

    try:
        
        #Compute distance:
        dist_2D = math.sqrt(pow(p2[0] - p1[0], 2) + pow(p2[1] - p1[1], 2))
    
    
    #Print error message if distance cannot be calculated:
    except:
        
        print("2D Euclidean distance could not be calculated...")
    
    #Return distance value:
    return dist_2D

In [26]:
#Compute obstruction angle between window centroid and 
#candidate obstruction point:
def get_OA_candidate_p(win_c, candidate_obstr_p):
    
    #Compute 2D distance between points:
    dist_2d = point_2D_dist((get_TOC_column(win_c,'C_X_coord', True)[0],
                             get_TOC_column(win_c,'C_Y_coord', True)[0]),
                            (candidate_obstr_p[1],
                             candidate_obstr_p[2]))
    
    #Compute 3D distance between points:
    dist_3d = point_3D_dist((get_TOC_column(win_c,'C_X_coord', True)[0],
                             get_TOC_column(win_c,'C_Y_coord', True)[0], 
                             get_TOC_column(win_c,'C_Z_coord', True)[0]),
                            (candidate_obstr_p[1],
                             candidate_obstr_p[2],
                             candidate_obstr_p[3]))
    
    #Compute height difference between points:
    h_diff = abs(candidate_obstr_p[3]-get_TOC_column(win_c,'C_Z_coord',True)[0])
    
    #If the window centroid and the candidate obstrucion point coincide:
    if ((h_diff==0) & (dist_2d==0)):
        
        #Obstruction angle is zero:
        oa = 0
        
        print("\nWindow centroid & candidate obstruction point have the same x-, y- & z-coordinates!")
    
    #If the window centroid & candidate obstrucion point have the same height:
    elif (h_diff==0):
        
        #Obstruction angle is zero:
        oa = 0
        
        print("\nWindow centroid & candidate obstruction point have the same height!")
    
    #If the 2D distance between the points is zero:
    elif (dist_2d==0):
        
        #Obstruction angle is zero:
        oa = 0
        
        print("\nWindow centroid & candidate obstruction point have the same 2D coordinates!")
    
    
    else:
        
        #Compute obstrucion angle:
        oa = math.degrees(math.atan((h_diff/dist_2d)))
    
    
    
    #Return tuple with obstruction angle & 3D distance between points (OA, 3D_dist): 
    return (oa, dist_3d)

In [27]:
#Function that takes a DSM, a window centroid and a 
#viewshed raster as input, computes the OA and returns
#a tuple (OA, NEAR_DIST) with the obstruction angle 
#value and the 2D-distance between the window centroid
#and the obstruction point with the maximum height:
def calc_OA(dsm, w_centroid, viewshed_raster, vcs):
    
    #Extract visible cells from DSM using the viewshed raster:
    visible_dsm = extract_by_mask(dsm, viewshed_raster)
    
    #Check if there are no visible cells:
    if(check_empty_raster(visible_dsm)):
        
        #Set obstruction angle value to NULL:
        oa = (None, None)
        print('\nNo visible cells...\nExtract DSM by mask viewshed returns empty raster.')
    
    #If there are visible cells:
    else:
        
        #Get window centroid z-coordinate:
        c_z_coord = get_TOC_column(w_centroid, 'C_Z_coord', True)[0]
        
        #Check if window centroid z-coordinate is valid:
        if(isinstance(c_z_coord, float)):
        
            #Filter out cells whose elevation is lower
            #than that of the window centroid:
            arcpy.CheckExtension('spatial') 
            arcpy.CheckOutExtension('spatial')
            obstructions = arcpy.sa.ExtractByAttributes(visible_dsm, "VALUE > "+ str(c_z_coord))
            arcpy.CheckInExtension('spatial')
            
            #Check if there are no obsruction cells with elevation 
            #higher than the height of of the window centroid:
            if (check_empty_raster(obstructions)):
                
                #If there are no such raster cells fullfilling the requirement OA=0:
                oa=(0, -1)
                
                print('\nViewshed DSM cells have lower elevation values than the height of the window centroid...')
            
            #If there are obstruction cells with a higher elevation that 
            #the height of the window centroid:
            else:
    
                #Convert raster with obstructions to points:
                arcpy.RasterToPoint_conversion(obstructions, 'obs_points.shp', 'Value')

                #Convert candidate obstruction points from 2D to 3D:
                arcpy.CheckExtension('3D') 
                arcpy.CheckOutExtension('3D')
                arcpy.FeatureTo3DByAttribute_3d('obs_points.shp', 'obs_points_3D.shp', 'grid_code')
                arcpy.CheckInExtension('3D')
                
                #Create fields to add x-, y- & z-coordinates:
                create_long_field('obs_points_3D.shp', "X_coord", "LONG", 12, 3)
                create_long_field('obs_points_3D.shp', "Y_coord", "LONG", 12, 3)
                create_long_field('obs_points_3D.shp', "Z_coord", "LONG", 12, 3)
                
                #Call function to add fields with x-,y-,z-coordinates:
                arcpy.management.CalculateGeometryAttributes(in_features='obs_points_3D.shp',
                                                             geometry_property=[["X_coord", "POINT_X"],
                                                                                ["Y_coord", "POINT_Y"],
                                                                                ["Z_coord", "POINT_Z"]],
                                                             length_unit="", area_unit="",
                                                             coordinate_system=arcpy.SpatialReference(3008, vcs))
                
                #Print number of candidate obstruction points:
                print('\nNumber of candidate points: ', len(get_TOC_column('obs_points_3D.shp', "FID", False)))
                    
                #Create list to store OA for all candidate obstruction points:
                OA_ls = []
                
                #Loop through all candidate obstruction points and compute their OA:
                with arcpy.da.SearchCursor('obs_points_3D.shp', ('FID', 'X_coord', 'Y_coord', 'Z_coord')) as c_obstr_pts:
                    
                    for cp in c_obstr_pts:
                        
                        #print("\nComputing OA for candidate obstruction point: ", cp[0])
                        
                        #Add OA-value for current candidate obsrucion point to list:
                        OA_ls.append(get_OA_candidate_p(w_centroid, cp))
                
                
                #print('\nOA-list: ', OA_ls)

                #Get max OA-value from list:
                oa = max(OA_ls, key = lambda i : i[0])

                print('\nSelected OA: ', oa)
        
        #If window centroid z-coordinate is not valid:
        else:
            #Set obstruction angle value to NULL:
            oa=(None, None)
            print('\nWindow centroid z-coordinate is not valid!')
            
        
        
    #Return obstruction angle:
    return oa

In [28]:
#Function that takes two strings as input 
#(fc_path = path to shapefile containing building footprints)
#(b_id: ID of builidng or building part) and returns the string 
#of a shapefile with the geometry of the selected building footprint:
#(Observe that shapefile is stored in working environment)
def get_b_fp(fc_path, b_ID):
    
    #Save information for current centroid to a separate layer:
    select_b_fp = arcpy.management.SelectLayerByAttribute(in_layer_or_view=fc_path,
                                                          selection_type="NEW_SELECTION",
                                                          where_clause="b_id = '"+str(b_ID)+"'", 
                                                          invert_where_clause="")
            
    #Write selected feature to a new feature class:
    save_as_fc(select_b_fp, 'b_fp.shp')
    
    #Return shapefile:
    return 'b_fp.shp'
    
    

In [29]:
#Function that checks if a viewshed raster overlaps a
#building footprint & returns a boolean value:
#TRUE  -- > viewshed overlaps building footprint
#FALSE -- > viewshed does not overlap building footprint
def vs_overlaps_b(vs, b_fp):
    
    try:
        
        #Convert viewshed from raster to polygon:
        arcpy.RasterToPolygon_conversion(vs, "vs_poly.shp", "NO_SIMPLIFY")
        
    except:
        
        print('\nError! could not convert raster to polygon...')
        
    
    #Check if the viewshed polygon & the building footprint have overlapping areas:
    try:
        
        #Execute intersection tool over viewshed polygon & building footprint polygon:
        arcpy.analysis.Intersect([b_fp, "vs_poly.shp"], "vs_intersect_b_fp.shp")
        
    except:
        
        print('\nError! Could not perform "intersect" over viewshed polygon & building footprint...')
        
    #Check if output shapefile is empty:
    if (shape_is_empty("vs_intersect_b_fp.shp")==True):
        
        return False
        
    else:
        
        return True
        
    
    

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="main_func"></a>

<br>
<br>

## Main function

In [ ]:
#Add column to window fc for storing obstruction angle value:
add_OA_field(w_fc)

#Read windows layer:
with arcpy.da.SearchCursor(w_fc, ('FID', 'Shape@', 'b_id', 'w_id')) as w_c, arcpy.da.SearchCursor(c_output, ('FID', 'Shape', 'C_Z_coord')) as c_c, arcpy.da.UpdateCursor(w_fc,('OA')) as upd_cursor:
    
    #Loop through every window:
    for w_row, c_row, upd_row in zip(w_c, c_c, upd_cursor):
        
        print('window_fid: ', w_row[0])
        print('window_centroid: ', c_row[0])
        print('window_id: ', w_row[3])
        
        #Extract only vertices for one window (will be removed in the end)
        #if(w_row[0]<=4):
        
        #Create and initialize variable to store window-centroid-id (corresponds to centroid-FID):
        w_c_id = w_row[0]
        
        #Extract top 2 window vertices of same height:
        w_top2_v = extract_rectangle_top_vertices(w_row[1])
        
        #Check that top 2 vertices from window have been extracted:
        if(w_top2_v is not None):
                
            #Call function to get suggestions for start- & end horizontal angle values: 
            hav1_ls, hav2_ls = get_viewshed_horiz_angles(w_top2_v)
            
            print()
            print(hav1_ls)
            print(hav2_ls)
            
            #Save information for current centroid to a separate layer:
            select_c = arcpy.management.SelectLayerByAttribute(in_layer_or_view=c_output,
                                                               selection_type="NEW_SELECTION",
                                                               where_clause="FID = "+str(w_c_id), 
                                                               invert_where_clause="")
            
            #Write selected features to a new feature class:
            save_as_fc(select_c, 'c.shp')
            
            ### Compute viewshes for both horizontal angle values ###
            #Add extension checkouts to ensure that the Viewshed2_3d-tool will run (license check): 
            arcpy.CheckExtension("3D") 
            arcpy.CheckOutExtension("3D")
            
            #Run Viewshed Analysis for selected window centroid:
            arcpy.Viewshed2_3d(in_raster = dsm_fc,
                               in_observer_features = 'c.shp',
                               out_raster = vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_A.tif",
                               analysis_type = "FREQUENCY",
                               refractivity_coefficient = 0.13,
                               observer_elevation = c_row[2], 
                               observer_offset = 0, 
                               horizontal_start_angle = hav1_ls[0],
                               horizontal_end_angle = hav1_ls[1], 
                               analysis_method = "ALL_SIGHTLINES")
            
            #Run Viewshed Analysis for selected window centroid:
            arcpy.Viewshed2_3d(in_raster = dsm_fc,
                               in_observer_features = 'c.shp',
                               out_raster = vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_B.tif",
                               analysis_type = "FREQUENCY",
                               refractivity_coefficient = 0.13,
                               observer_elevation = c_row[2], 
                               observer_offset = 0, 
                               horizontal_start_angle = hav2_ls[0],
                               horizontal_end_angle = hav2_ls[1], 
                               analysis_method = "ALL_SIGHTLINES")
            
            #Add extension checkouts to ensure that the Viewshed2_3d-tool will run (license check):
            arcpy.CheckInExtension("3D")
            
            #Save path for viewshed raster A & B:
            vs_A = vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_A.tif"
            vs_B = vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_B.tif"
            
            #Check if viewshed-rasters are both not empty:
            if((check_empty_raster(vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_A.tif")==False) & 
               (check_empty_raster(vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_B.tif")==False)):
                
                #Call function to compute obstruction angle for both viewshed rasters:
                OA_A = calc_OA(dsm_fc, 'c.shp', vs_A, VCS)
                OA_B = calc_OA(dsm_fc, 'c.shp', vs_B, VCS)
                
                #Get building/bulding part footprint that corresponds to the 
                #building id of the current window:
                b_fp_2D = get_b_fp(b_fp_fc, w_row[2])
                
                print('\nNone of the viewshed rasters is empty!')
                print('OA_A: ', OA_A)
                print('OA_B: ', OA_B)
                
                #Check if obstruction angles from both directions are empty: 
                if((OA_A[0] is None) & (OA_B[0] is None)):
                    
                    #Set window obstrucion angle value to None:
                    OA = -9999
                    
                    print('\nBoth obsruction angles are None!')
                
                #Check if one of the returned obstruction angles is None: 
                elif ((OA_A[0] is not None) & (OA_B[0] is None)):
                    
                    print('\nObsruction angle B is None!')
                    
                    #Make sure that the viewshed of the OA is not overlapping the builing footprint:
                    if(vs_overlaps_b(vs_A, b_fp_2D)):
                        
                        #If viewshed of A overlaps building footprint:
                        OA = -9999
                        
                        print('\nViewshed A overlaps with building footprint!')
                    
                    #If viewshed of A does not overlap building footprint:   
                    else:
                        
                        #Set window obstrucion angle value:
                        OA = OA_A[0]
                    
                    
                    
                #Check if one of the returned obstruction angles is None: 
                elif ((OA_A[0] is None) & (OA_B[0] is not None)):
                    
                    print('\nObsruction angle A is None!')
                    
                    #Make sure that the viewshed of the OA is not overlapping the builing footprint:
                    if(vs_overlaps_b(vs_B, b_fp_2D)):
                        
                        #If viewshed of B overlaps building footprint:
                        OA = -9999
                        
                        print('\nViewshed B overlaps with building footprint!')
                    
                    #If viewshed of A overlaps building footprint:   
                    else:
                        
                        #Set window obstrucion angle value:
                        OA = OA_B[0]                    
                  
                
                #If both computed obstruction angles are not None:
                else:
                    
                    print('\nNone of the obsruction angles is None!')
                    
                    #If wiewshed-A overlaps with the building footprint:
                    if(vs_overlaps_b(vs_A, b_fp_2D)):
                        
                        #Pick obstruction angle calculated for viewshed B:
                        OA = OA_B[0]
                        
                        print('\nViewshed A overlaps with building footprint!')
                    
                    #If viewshed-B overlaps with the building footprint:
                    elif(vs_overlaps_b(vs_B, b_fp_2D)):
                        
                        #Pick obstruction angle calculated for viewshed A:
                        OA = OA_A[0]
                        
                        print('\nViewshed B overlaps with building footprint!')
                    
                    #If none of the viewsheds overlap with the building footprint:
                    else:
                        
                        print('\nViewsheds do not overlap with building footprint!')
                        print('\nOA is determined by obstacle that is furthes away...')
                        
                        #Check distance between window centroid & obstruction point:
                        if (OA_A[1]<OA_B[1]):

                            #Choose the obstruction angle of the pair with the largest 2D distance:
                            OA = OA_B[0]

                        elif (OA_A[1]>OA_B[1]):

                            #Choose the obstruction angle of the pair with the largest 2D distance:
                            OA = OA_A[0]

                        else:

                            #Set window obstrucion angle value to an error value:
                            OA = -9999
                    
                    
                    
                
            
            #Check if one viewshed-raster has values and the other is empty (case 1):
            elif((check_empty_raster(vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_A.tif")==False) & 
                 (check_empty_raster(vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_B.tif")==True)):
                
                #Call function to compute obstruction angle:
                OA_tuple = calc_OA(dsm_fc, 'c.shp', vs_A, VCS)
                
                #Check that OA is not None:
                if (OA_tuple[0] is not None):
                
                    #Exract obstruction angle value:
                    OA = OA_tuple[0]
                
                #If OA is None:    
                else:
                    
                    OA = -9999
                
                print('\nViewshed raster B is empty!')
                print('OA_tuple: ', OA_tuple)
                print('OA: ', OA)
                
            #Check if one viewshed-raster has values and the other is empty (case 2):    
            elif((check_empty_raster(vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_B.tif")==False) & 
                 (check_empty_raster(vs_path+"vs_90_raster_b"+str(b_number)+"_"+str(w_row[3])+"_A.tif")==True)):
                
                #Call function to compute obstruction angle:
                OA_tuple = calc_OA(dsm_fc, 'c.shp', vs_B, VCS)
                
                #Check that OA is not None:
                if (OA_tuple[0] is not None):
                
                    #Exract obstruction angle value:
                    OA = OA_tuple[0]
                
                #If OA is None:    
                else:
                    
                    OA = -9999
                
                print('\nViewshed raster A is empty!')
                print('OA_tuple: ', OA_tuple)
                print('OA: ', OA)
                
            #Both viewshed rasters are empty:   
            else:
                #OA = None (ArcPy does not except None in OA even it is set as nullable...)
                OA = -9999
                print('Viewshed rasters are empty for both perpendicular directions of this window...')
                
            
            #Update value in OA-column of window fc with computed obstruction angle:
            upd_row[0] = OA
            upd_cursor.updateRow(upd_row)
            
            #Delete temporary fields from window table:
            arcpy.management.DeleteField('c.shp', 'NEAR_FID')
            arcpy.management.DeleteField('c.shp', 'NEAR_DIST')
            arcpy.management.DeleteField('c.shp', 'NEAR_DIST3')
            arcpy.management.DeleteField('c.shp', 'NEAR_ANG_V')
            arcpy.management.DeleteField('c.shp', 'NEAR_ANG_H')
            
            #Delete intermediate files from workspace folder:
            delete_dir_files(working_environment)
            
            
            
            print('\nObstruction angle (OA): ', OA)
            
            
        else:
            print('Failed to extract top 2 vertices from window geometry...')
            continue    
            
        
        
            
            
            
            

        
        print()
        print()
        print()
        print()
        print()

window_fid:  0
window_centroid:  0
window_id:  w1

pair 1:  [228.5461046234, 218.546104623]
pair 2:  [38.5461046234, 48.54610462299999]

[218.546104623, 228.5461046234]
[38.5461046234, 48.54610462299999]

Number of candidate points:  119

Selected OA:  (4.699969104638084, 32.097450802393894)

Viewshed raster A is empty!
OA_tuple:  (4.699969104638084, 32.097450802393894)
OA:  4.699969104638084

Obstruction angle (OA):  4.699969104638084





window_fid:  1
window_centroid:  1
window_id:  w2

pair 1:  [228.5461045679, 218.546104568]
pair 2:  [38.54610456789999, 48.546104568000004]

[218.546104568, 228.5461045679]
[38.54610456789999, 48.546104568000004]

Number of candidate points:  119

Selected OA:  (10.155192537801112, 32.49865147990138)

Viewshed raster A is empty!
OA_tuple:  (10.155192537801112, 32.49865147990138)
OA:  10.155192537801112

Obstruction angle (OA):  10.155192537801112





window_fid:  2
window_centroid:  2
window_id:  w3

pair 1:  [228.5461045679, 218.546104568]
pair 2

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="exe_code"></a>

<br>
<br>

## Execute calculations

<br>
<br>

In [ ]:
# Get description of input feature class:
#desc = arcpy.da.Describe(w_fc)

In [ ]:
#Call function to compute window centroids:
#compute_w_centroid(w_fc, VCS, c_output)

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="control_out"></a>

<br>
<br>

## Control output

In [ ]:
#Check window shapefile columns:
get_field_names(w_fc)

In [ ]:
#Check window shapefile attribute table (incl. OA results):
get_TOC(w_fc)

In [ ]:
#Set building and window id for checking
b_id = 1
w_id = 7

#Set path to raster file:
raster_file = vs_path+"vs_90_raster_b"+str(b_id)+"_w"+str(w_id)+"_B.tif"

#Check if raster is empty:
check_empty_raster(raster_file)

In [ ]:
#Read raster:
raster = arcpy.Raster(raster_file)
    
#Get raster maximum value:
max_val = arcpy.sa.Raster(raster).maximum

max_val

In [ ]:
#arcpy.da.Describe(dsm_fc)

In [ ]:
get_field_names(input_feature_class)

In [ ]:
get_TOC(input_feature_class)

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>
<a id="references"></a>
<br>
<br>

## Reference links:

- Extensions:
    - https://gis.stackexchange.com/questions/54044/arcgis-error-000824-the-tool-is-not-licensed-in-arcpy
    - https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/checkextension.htm
    
    
- Feature vertices to Points (Data Management):
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/feature-vertices-to-points.htm


- Add Z Information (3D Analyst):
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/3d-analyst/add-z-information.htm


- Centroid calculation (alternative for 2D datasets):
    - https://stackoverflow.com/questions/23020659/fastest-way-to-calculate-the-centroid-of-a-set-of-coordinate-tuples-in-python-wi
    
    
- Spatial Reference:
    - https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/spatialreference.htm
    

- Delete Shapefile:
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/delete.htm
    
    
- Find files in directory:
    - https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python
    
    
- Join Field (Data Management):
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/join-field.htm
    

- Viewshed 2 (3D Visibility toolset):
    - https://desktop.arcgis.com/en/arcmap/latest/tools/3d-analyst-toolbox/viewshed-2.htm
    
    
- [Raster to Polygon](https://pro.arcgis.com/en/pro-app/2.8/tool-reference/conversion/raster-to-polygon.htm)


- [Intersect](https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/intersect.htm)


- [Check if shapefile is empty](https://gis.stackexchange.com/questions/71197/how-to-check-if-shapefile-is-empty-using-arcpy)

<div style="text-align: right">
<a href="#intro">Back to top</a>
</div>

<br>
<br>